In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('final_data.csv')

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,...,age,python_yn,R_yn,spark,aws,excel,job_simp,seniority,desc_len,num_comp
0,0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",6,...,47,1,0,0,0,1,2,1,2536,0
1,1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",2,...,36,1,0,0,0,0,2,1,4783,0
2,2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",6,...,10,1,0,1,0,1,2,1,3461,0
3,3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",3,...,55,1,0,0,0,0,2,1,3883,3
4,4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",7,...,22,1,0,0,0,1,2,1,2728,3


In [4]:
corr_matrix = df.corr()

In [5]:
corr_matrix['avg_salary'].sort_values(ascending=False)

avg_salary           1.000000
max_salary           0.986862
min_salary           0.967765
seniority            0.349392
python_yn            0.327796
spark                0.176559
aws                  0.171379
num_comp             0.093813
desc_len             0.078808
employer_provided    0.076287
age                  0.019655
Rating               0.013492
Revenue             -0.007132
Founded             -0.015149
Size                -0.019268
same_state          -0.034641
R_yn                -0.040000
Unnamed: 0          -0.041765
Unnamed: 0.1        -0.041765
job_simp            -0.051373
excel               -0.057252
Sector              -0.067098
Industry            -0.103017
Type of ownership   -0.126853
job_state           -0.186570
hourly              -0.356082
Name: avg_salary, dtype: float64

In [6]:
df_final = df[['avg_salary','job_simp','excel','Sector','Industry','Type of ownership','job_state','hourly','seniority','python_yn','spark','aws','age','Rating']]

In [7]:
df_final.head()

,avg_salary,job_simp,excel,Sector,Industry,Type of ownership,job_state,hourly,seniority,python_yn,spark,aws,age,Rating
0,72.0,2,1,2,3,2,24,0,1,1,0,0,47,3.8
1,87.5,2,0,12,28,7,17,0,1,1,0,0,36,3.4
2,85.0,2,1,6,46,2,7,0,1,1,1,0,10,4.8
3,76.5,2,0,19,18,4,35,0,1,1,0,0,55,3.8
4,114.5,2,1,6,2,2,25,0,1,1,0,0,22,2.9


In [8]:
from sklearn.model_selection import train_test_split

x =df_final.drop('avg_salary', axis=1)
y = df_final["avg_salary"].copy()
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [9]:
#from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score

lm = LinearRegression()
lm.fit(x_train, y_train)

np.mean(cross_val_score(lm, x_train, y_train, scoring="neg_mean_absolute_error", cv=5))

lml = Lasso()

alpha = []
error = []

for i in range(1,100):
    alpha.append(i/100)
    lml = Lasso(alpha=(i/100))
    error.append(np.mean(cross_val_score(lml, x_train, y_train, scoring="neg_mean_absolute_error", cv=5)))
plt.plot(alpha,error)

err = tuple(zip(alpha,error))
df_err = pd.DataFrame(err, columns = ['alpha','error'])
df_err[df_err.error == max(df_err.error)]

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [11]:
rf.fit(x_train, y_train)

RandomForestRegressor()

In [12]:
np.mean(cross_val_score(rf, x_train, y_train, scoring="neg_mean_absolute_error", cv=5))

-15.930493483077734

In [13]:
some_data = x_train.iloc[:5]

In [14]:
some_labels = y_train.iloc[:5]

In [15]:
rf.predict(some_data)

array([ 84.745     ,  68.585     , 188.48      ,  93.75866667,
        72.695     ])

In [16]:
some_labels

380     87.5
500     64.5
176    225.0
571     91.5
642     69.5
Name: avg_salary, dtype: float64

In [18]:
from sklearn.metrics import mean_squared_error
rf.predict(x_test)
np.mean(cross_val_score(rf, x_test, y_test, scoring="neg_mean_absolute_error", cv=10))

-21.454605961791835

In [21]:
from joblib import dump, load
dump(rf, 'Final.sav')

['Final.sav']